# Peek at the database tables

In [3]:
import sqlite3
con = sqlite3.connect("interview.db").cursor()
# find table names
tables = con.execute("select name from sqlite_master")
# tables.fetchall()

# explore tables
for tablename in tables.fetchall():
    tablename = tablename[0]
    print(tablename)
    cols = con.execute(f"select name from pragma_table_info('{tablename}')").fetchall()

    print(cols)
    for row in con.execute(f"select * from {tablename} limit 5").fetchall():
        print(row)
con.close()

model_scores_by_zip
[('zcta',), ('state_code',), ('state name',), ('neighborhood_stress_score',), ('algorex_sdoh_composite_score',), ('social_isolation_score',), ('transportation_access_score',), ('food_access_score',), ('unstable_housing_score',), ('state_govt_assistance',), ('homeless_indicator',), ('derived_indicator',)]
(94720, 6.0, 'California', -0.31, 6.19, 2.37, 3.89, 3.06, 2.03, 0.59, 10.0, 1)
(95675, 6.0, 'California', -0.31, 6.18, 3.75, 4.42, 3.9, 2.76, 0.33, 6.0, 1)
(95699, 6.0, 'California', -0.31, 6.18, 3.75, 4.42, 3.9, 2.76, 0.33, 6.0, 1)
(95930, 6.0, 'California', 0.33, 6.0, 3.47, 3.95, 3.01, 3.05, 0.76, 10.0, 1)
(95941, 6.0, 'California', 0.33, 6.0, 3.47, 3.95, 3.01, 3.05, 0.76, 10.0, 1)
roster_1
[('Person_Id',), ('First_Name',), ('Last_Name',), ('Dob',), ('Age',), ('Gender',), ('Street_Address',), ('State',), ('City',), ('Zip',), ('eligibility_start_date',), ('eligibility_end_date',), ('payer',)]
('15340001', 'Daniel', 'Smith', '2017-04-27', '5', 'Male', '1505 Alvarez 

# Creating std_member_info and standardization of member rosters 

In [2]:
import sqlite3
import queries
from run import build_std_member_info, build_std_member_info_no_dupe

con = sqlite3.connect("interview.db")
# assume that we don't want to eliminate dupes yet
build_std_member_info(con)
# build_std_member_info_no_dupe(con) #if we do run this

queries.num_distinct_members_apr2022(con)
queries.duplicate_members(con)

# using no duplicate data for all 
# assume everyone has a score (no null zip/score/non join)
queries.members_by_payer(con)
queries.food_access_score_lt2(con)
queries.avg_social_isolation_score(con)
# assume we want the member_ids 
queries.highest_algorex_sdoh_composite_score(con)

con.close()

distinct member_ids: [(86418,)]
duplicate member_ids: [(22795,)]
number members by payer: [('Madv', 32347), ('Mdcd', 54071)]
number members with food_access_score < 2: [(6676,)]
average social_isolation_score: [(3.067700594783634,)]
members living in zip with highest algorex_sdoh_composite_score: [(38, '95950', 8.77)]


# Email to Project Manager

Hey PM,

I just finished aggregating and standardizing the member rosters. The sqlite table is located at `./interview.db` under the name `std_member_info` with column names `member_id, member_first_name, member_last_name, date_of_birth, main_address, city, state, zip_code, payer`.

Here is are the summary statistics you requested. 
- Of the members in the 5 rosters, 86418/109213 members were distinct and 22795/109213 were duplicate. 
- By payer, 32347 distinct members were 'Madv' and 54071 distinct members were 'Mdcd'

These queries were done under the assumption that each distinct member had a zipcode with a corresponding score located in the `model_scores_by_zip` table.
- There were 6676 distinct members with a `food_access_score` less than 2. 
- The average `social_isolation_score` was 3.068. 
- There were 38 members living in the zipcode with the highest `algorex_sdoh_composite_score`. This zipcode was 95950, and the SDOH score was 8.77. If you would like specific `member_id` of the people living in this zipcode, please let me know. 


To get these results, I first created the `std_member_info` table with the specified column names. Then for each member roster table, I took a peek at the first few rows to figure out the format of the data. These are some observations I had:
1) The columns of the rosters are named differently (Person_Id, First_Name, Last_Name, Dob, Street_Address, City, State, Zip, payer). 
2) In `roster_4`, the state is represented by the state abbreviation 'CA' instead of 'California'. This differs from the other 4 rosters as they all use the full state name. I assumed that full state names were desired, so I converted the abbreviations in this roster to be full state names.
3) In `roster_2` the date is formatted as `mm/dd/yyyy` instead of `yyyy-mm-dd` which is the format that the other 4 rosters use. I assumed that the format that is most common was desired so I converted the dates in this roster to be formatted as `yyyy-mm-dd`.

After looking at the rosters, I inserted them 1 by 1 into the `std_member_info` table. This allows for duplicate member_ids to be added to the table if any members are in more than 1 roster table. I have also made a version of this script to skip any duplicate member_ids so that the table is duplicate free. Let me know which version you prefer. 

With the table created, I ran the queries that you requested. Please let me know if you would like me to explain how the queries work in further detail.

Best,

Albert
